In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import pvlib
from pvlib.pvsystem import PVSystem, Array, FixedMount
from pvlib.location import Location
from pvlib.modelchain import ModelChain
import time

sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
sapm_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')
module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']
inverter = sapm_inverters['ABB__MICRO_0_25_I_OUTD_US_208__208V_']
temperature_model_parameters = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']

c:\Users\lilia\AppData\Local\Programs\Python\Python39\lib\site-packages\geopandas\_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [5]:
def get_weather(lat, lng):
    print('Getting weather...')
    start = time.time()

    weather = pvlib.iotools.get_pvgis_tmy(lat, lng,
                                        map_variables=True)[0]
    weather.index.name = "utc_time"
    
    end = time.time()
    print(f'Completed getting weather in {end-start}s')
    return weather

In [6]:
def calculate_PV(db):
    coordinates = list(zip(db['lat'], db['lng'], db['height_mean'], db['slope_mean'], db['aspect_mean'], db['shading_mean']))
    timezone = 'Etc/GMT+1'

    energies = []
    for location in coordinates:
        print('Calculating solar output...')
        latitude, longitude, height, slope, aspect, shading = location
        location = Location(
            latitude,
            longitude,
            name='',
            altitude=height,
            tz=timezone,
        )
        mount = FixedMount(surface_tilt=slope, surface_azimuth=aspect)
        array = Array(
            mount=mount,
            module_parameters=module,
            temperature_model_parameters=temperature_model_parameters,
        )
        system = PVSystem(arrays=[array], inverter_parameters=inverter)
        mc = ModelChain(system, location)
        mc.run_model(weather)
        annual_energy = mc.results.ac.sum()
        energies.append(annual_energy * shading)

        print(f"Completed calculating PV output {end-start}s")
    
    db['pv_output'] = energies
    return db

In [7]:
db = gpd.read_file('output/sp0585_DSM_1M.gml', driver='GML')
db = db.to_crs(4326)
db['lng'] = db.geometry.centroid.x
db['lat'] = db.geometry.centroid.y
# db['pv_output'] = calculate_PV(db)
db.head()

C:\Users\lilia\AppData\Local\Temp\ipykernel_14896\1394035029.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  db['lng'] = db.geometry.centroid.x
C:\Users\lilia\AppData\Local\Temp\ipykernel_14896\1394035029.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  db['lat'] = db.geometry.centroid.y


,gml_id,fid,slope,fid_2,aspect,slope_mean,aspect_mean,height_mean,shading_mean,AREA,geometry,lng,lat
0,sp0585_DSM_1M.77,18,1,61,3,0.650338,181.044024,167.892334,0.916667,22.149597,"MULTIPOLYGON (((-1.92678 52.47099, -1.92678 52...",-1.926724,52.470977


In [8]:
db = gpd.read_file('output_osmp/SJ9000.gml', driver='GML')
db = db.to_crs(4326)
db['lng'] = db.geometry.centroid.x
db['lat'] = db.geometry.centroid.y
db.rename(columns={'AbsHMax':'height_mean'}, inplace=True)

print('Adding columns...')
start = time.time()
if 'slope_mean' not in db:
    db['slope_mean'] = [0] * len(db)
if 'aspect_mean' not in db:
    db['aspect_mean'] = [180] * len(db)
end = time.time()
print(f'Completed adding columns in {end-start}s')

latitude = db['lat'].mean()
longitude = db['lng'].mean()
weather = get_weather(latitude, longitude)

C:\Users\lilia\AppData\Local\Temp\ipykernel_14896\3002544429.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  db['lng'] = db.geometry.centroid.x
C:\Users\lilia\AppData\Local\Temp\ipykernel_14896\3002544429.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  db['lat'] = db.geometry.centroid.y


Adding columns...
Completed adding columns in 0.0249326229095459s
Getting weather...
Completed getting weather in 3.029014825820923s


In [10]:
db.columns

Index(['gml_id', 'fid', 'featureCode', 'versionDate', 'calculatedAreaValue',
       'RelH2', 'RelHMax', 'AbsHMin', 'height_mean', 'AbsH2', 'lastUpdateDate',
       'entryDate', 'uprn', 'country', 'multiOccCount', 'buildingNumber',
       'thoroughfare', 'postTown', 'postcode', 'level', 'shading_mean',
       'geometry', 'lng', 'lat', 'slope_mean', 'aspect_mean'],
      dtype='object')

In [105]:
print('Begin batch...')
start =time.time()

import multiprocessing as mp
# from pathos.pools import ParallelPool as Pool
from pathos.pools import ParallelPool as Pool

cores = mp.cpu_count()

df_split = np.array_split(db, cores, axis=0)

# create the multiprocessing pool
pool = Pool(cores)

# process the DataFrame by mapping function to each df across the pool
db_out = np.vstack(pool.map(calculate_PV, df_split))

# close down the pool and join
pool.close()
pool.join()
pool.clear()

end = time.time()
print(f"Completed batch in {end-start}s")

db_out.head()

Begin batch...


In [40]:
split_db = np.array_split(db, 50)
joined_db = gpd.GeoDataFrame(columns = db.columns, geometry='geometry')

for df in split_db:
    df['pv_output'] = calculate_PV(df)
    joined_db = gpd.GeoDataFrame(pd.concat([joined_db, df], ignore_index=True))

Calcuating PV output...
Completed calculating PV output in 890.1219239234924s
Calcuating PV output...
Completed calculating PV output in 870.1520748138428s
Calcuating PV output...
Completed calculating PV output in 890.803005695343s
Calcuating PV output...
Completed calculating PV output in 843.7936589717865s
Calcuating PV output...
Completed calculating PV output in 917.1566498279572s
Calcuating PV output...
Completed calculating PV output in 889.4253227710724s
Calcuating PV output...
Completed calculating PV output in 854.667886018753s
Calcuating PV output...
Completed calculating PV output in 887.2818069458008s
Calcuating PV output...
Completed calculating PV output in 872.7420446872711s
Calcuating PV output...
Completed calculating PV output in 849.3490014076233s
Calcuating PV output...
Completed calculating PV output in 848.4422409534454s
Calcuating PV output...
Completed calculating PV output in 866.8500733375549s
Calcuating PV output...
Completed calculating PV output in 857.663

KeyboardInterrupt: 

In [41]:
joined_db.to_file('output_osmp/SJ9000_results.gml', driver='GML')

c:\Users\lilia\AppData\Local\Programs\Python\Python39\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [67]:
split_db2 = np.array_split(db[17652:], 50)
joined_db2 = gpd.GeoDataFrame(columns = db.columns, geometry='geometry')

for df in split_db2:
    df['pv_output'] = calculate_PV(df, weather)
    joined_db2 = gpd.GeoDataFrame(pd.concat([joined_db2, df], ignore_index=True))

Calcuating PV output...
Calculating solar output...
Completed calculating PV output in 52.742833375930786s
Calcuating PV output...
Calculating solar output...
Completed calculating PV output in 50.5772647857666s
Calcuating PV output...
Calculating solar output...
Completed calculating PV output in 52.73547053337097s
Calcuating PV output...
Calculating solar output...
Completed calculating PV output in 56.75181603431702s
Calcuating PV output...
Calculating solar output...
Completed calculating PV output in 46.74575877189636s
Calcuating PV output...
Calculating solar output...
Completed calculating PV output in 44.91084599494934s
Calcuating PV output...
Calculating solar output...
Completed calculating PV output in 49.34625959396362s
Calcuating PV output...
Calculating solar output...
Completed calculating PV output in 52.44689655303955s
Calcuating PV output...
Calculating solar output...
Completed calculating PV output in 54.2777419090271s
Calcuating PV output...
Calculating solar outpu

KeyboardInterrupt: 